In [428]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn import set_config

set_config(transform_output='pandas')

In [429]:
df = pd.read_csv('../data/raw/train.csv')
df = pd.read_csv('../data/raw/test.csv')

train_df = pd.read_csv('/Users/kubrademirhan/Documents/kaggle_house_prices/data/raw/train.csv')
test_df = pd.read_csv('/Users/kubrademirhan/Documents/kaggle_house_prices/data/raw/test.csv')

In [430]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [431]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [432]:
X = train_df.drop(columns=['SalePrice'])
y = train_df.SalePrice

In [433]:
# Baseline Sanity Check
print('Train shape:', train_df.shape)
print('Test Shape:', test_df.shape)
print('Missing Value in train:', train_df.isna().sum().sum())
print('Missing Value in test:', test_df.isna().sum().sum())

Train shape: (1460, 81)
Test Shape: (1459, 80)
Missing Value in train: 7829
Missing Value in test: 7878


In [434]:
#Baseline Score
y.value_counts(normalize=True).values.max().round(2)*100

np.float64(1.0)

In [435]:
# Split + evaluation
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

In [436]:
from sklearn.pipeline import make_pipeline

num_pipe = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler()
)

num_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [437]:
cat_pipe = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(handle_unknown='ignore', sparse_output=False)
)

cat_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [438]:
preprocess_pipe = ColumnTransformer(transformers=[
    ('num', num_pipe, make_column_selector(dtype_include=np.number)),
    ('cat', cat_pipe, make_column_selector(dtype_exclude=np.number)),
])

preprocess_pipe

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x12e552850>),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x12e552d30>)])

In [439]:
preprocess_pipe.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x12e552850>),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x12e552d30>)])

In [440]:
preprocess_pipe.transform(X_train)

,num__Id,num__MSSubClass,num__LotFrontage,num__LotArea,num__OverallQual,num__OverallCond,num__YearBuilt,num__YearRemodAdd,num__MasVnrArea,num__BsmtFinSF1,...,cat__SaleType_ConLw,cat__SaleType_New,cat__SaleType_Oth,cat__SaleType_WD,cat__SaleCondition_Abnorml,cat__SaleCondition_AdjLand,cat__SaleCondition_Alloca,cat__SaleCondition_Family,cat__SaleCondition_Normal,cat__SaleCondition_Partial
254,-1.119284,-0.866764,-0.012468,-0.212896,-0.820445,0.372217,-0.455469,-1.346063,-0.597889,1.037269,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1066,0.790464,0.074110,-0.502357,-0.265245,-0.088934,1.268609,0.718609,0.439214,-0.597889,-0.971996,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
638,-0.216152,-0.631546,-0.146074,-0.177841,-0.820445,1.268609,-1.988293,-1.683818,-0.597889,-0.971996,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
799,0.162505,-0.161109,-0.457822,-0.324474,-0.820445,1.268609,-1.107734,-1.683818,0.861522,0.267995,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
380,-0.822944,-0.161109,-0.903175,-0.529035,-0.820445,0.372217,-1.531707,-1.683818,-0.597889,-0.496920,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,0.858669,-0.866764,0.343814,-0.127631,-0.088934,-0.524174,1.142582,1.018222,-0.597889,-0.919694,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1130,0.940986,-0.161109,-0.235145,-0.268313,-1.551955,-2.316957,-1.401254,-1.683818,-0.597889,0.383495,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1294,1.326699,-0.866764,-0.457822,-0.234096,-0.820445,1.268609,-0.520696,0.246211,-0.597889,-0.608062,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
860,0.305971,-0.161109,-0.680498,-0.283376,0.642577,2.165000,-1.727387,0.632217,-0.597889,-0.971996,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [441]:
preprocess_pipe.transform(X_train).isna().sum()

num__Id                       0
num__MSSubClass               0
num__LotFrontage              0
num__LotArea                  0
num__OverallQual              0
                             ..
cat__SaleCondition_AdjLand    0
cat__SaleCondition_Alloca     0
cat__SaleCondition_Family     0
cat__SaleCondition_Normal     0
cat__SaleCondition_Partial    0
Length: 286, dtype: int64

In [442]:
# Modeling

full_pipeline = make_pipeline(preprocess_pipe, Ridge())
full_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x12e552850>),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x12e552d30>)])),
                ('ridge', Ridge())])

In [443]:
full_pipeline.fit(X_train, y_train)
pred = full_pipeline.predict(X_test)

/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_pri

In [444]:
rmse = np.sqrt(mean_squared_error(y_test, pred))
print('RMSE:', rmse)

RMSE: 29840.03594796511


In [445]:
# Fine Tuning
full_pipeline.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('num',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='median')),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x12e552850>),
                                   ('cat',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='most_frequent')),
                                                    ('onehotencoder',
                                                     OneHotEncoder(handle_unknown='ignore',
                                                                   sparse_output=False))]),
                       

In [446]:
from sklearn.model_selection import GridSearchCV


grid = {'ridge__alpha': [0.1, 1.0, 10.0, 100.0]}

search = GridSearchCV(full_pipeline, param_grid=grid, cv=5, scoring='neg_root_mean_squared_error')
search

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x12e552850>),
                                                                        ('cat',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x12e552d30>)])),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': [0.1, 1.0, 10.0, 100.0]},
             scoring='neg_root_mean_squared_error')

In [447]:
search.fit(X_train, y_train)

/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_pri

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x12e552850>),
                                                                        ('cat',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x12e552d30>)])),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': [0.1, 1.0, 10.0, 100.0]},
             scoring='neg_root_mean_squared_error')

In [448]:
search.best_params_

{'ridge__alpha': 10.0}

In [449]:
search.best_score_

np.float64(-32580.023832898372)

In [450]:
best_model = search.best_estimator_
pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, pred))
rmse

/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


np.float64(30652.70365337626)

In [451]:
from sklearn.linear_model import ElasticNet

full_pipe_en = make_pipeline(
    preprocess_pipe,
    ElasticNet(max_iter=20000, random_state=42)
)

full_pipe_en

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x12e552850>),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x12e552d30>)])),
                ('elasticnet', ElasticNet(max_iter=20000, random_state=42))])

In [452]:
param_grid_en = {
    'elasticnet__alpha': [0.01, 0.1, 1.0, 10.0, 50.0],
    'elasticnet__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
}

search_en = GridSearchCV(
    estimator=full_pipe_en,
    param_grid=param_grid_en,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    error_score='raise',
    verbose=2
)

search_en

GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x12e552850>),
                                                                        ('cat',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          Si...
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x12e552d30>)])),
                                       ('elasticnet',
                                        ElasticNet(max_iter=20000,
                                                   random_state=42))]),
             n_jobs=-1,
             param_grid={'elasticnet__alpha': [0.01, 0.1, 1.0, 10.0, 50.0],
                         'elasticnet__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]},
             scoring='neg_root_mean_squared_error', verbose=2)

In [453]:
search_en.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul


[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.1; total time=   0.2s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.1; total time=   0.3s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.1; total time=   0.2s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.1; total time=   0.2s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.1; total time=   0.2s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.3; total time=   0.2s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.3; total time=   0.2s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.3; total time=   0.2s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.3; total time=   0.3s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.3; total time=   0.3s


/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul


[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.5; total time=   0.3s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.5; total time=   0.3s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.5; total time=   0.3s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.5; total time=   0.3s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.5; total time=   0.3s
[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.7; total time=   0.3s


...

[CV] END ...elasticnet__alpha=0.01, elasticnet__l1_ratio=0.9; total time=   1.0s
[CV] END ....elasticnet__alpha=1.0, elasticnet__l1_ratio=0.5; total time=   0.1s
[CV] END ....elasticnet__alpha=1.0, elasticnet__l1_ratio=0.7; total time=   0.1s
[CV] END ....elasticnet__alpha=1.0, elasticnet__l1_ratio=0.7; total time=   0.1s
[CV] END ....elasticnet__alpha=1.0, elasticnet__l1_ratio=0.7; total time=   0.1s
[CV] END ....elasticnet__alpha=1.0, elasticnet__l1_ratio=0.7; total time=   0.1s
[CV] END ....elasticnet__alpha=1.0, elasticnet__l1_ratio=0.9; total time=   0.1s
[CV] END ....elasticnet__alpha=1.0, elasticnet__l1_ratio=0.9; total time=   0.1s
[CV] END ....elasticnet__alpha=1.0, elasticnet__l1_ratio=0.7; total time=   0.1s
[CV] END ...elasticnet__alpha=10.0, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END ....elasticnet__alpha=1.0, elasticnet__l1_ratio=0.9; total time=   0.1s
[CV] END ....elasticnet__alpha=1.0, elasticnet__l1_ratio=0.9; total time=   0.1s
[CV] END ...elasticnet__alph

/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
 

[CV] END ...elasticnet__alpha=10.0, elasticnet__l1_ratio=0.5; total time=   0.1s
[CV] END ...elasticnet__alpha=10.0, elasticnet__l1_ratio=0.5; total time=   0.1s
[CV] END ...elasticnet__alpha=10.0, elasticnet__l1_ratio=0.7; total time=   0.0s
[CV] END ...elasticnet__alpha=10.0, elasticnet__l1_ratio=0.5; total time=   0.1s
[CV] END ...elasticnet__alpha=10.0, elasticnet__l1_ratio=0.7; total time=   0.1s
[CV] END ...elasticnet__alpha=10.0, elasticnet__l1_ratio=0.9; total time=   0.0s
[CV] END ...elasticnet__alpha=10.0, elasticnet__l1_ratio=0.7; total time=   0.1s
[CV] END ...elasticnet__alpha=10.0, elasticnet__l1_ratio=0.7; total time=   0.1s
[CV] END ...elasticnet__alpha=10.0, elasticnet__l1_ratio=0.7; total time=   0.1s
[CV] END ...elasticnet__alpha=10.0, elasticnet__l1_ratio=0.9; total time=   0.1s
[CV] END ...elasticnet__alpha=10.0, elasticnet__l1_ratio=0.9; total time=   0.1s
[CV] END ...elasticnet__alpha=50.0, elasticnet__l1_ratio=0.1; total time=   0.0s
[CV] END ...elasticnet__alph

/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul


GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x12e552850>),
                                                                        ('cat',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          Si...
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x12e552d30>)])),
                                       ('elasticnet',
                                        ElasticNet(max_iter=20000,
                                                   random_state=42))]),
             n_jobs=-1,
             param_grid={'elasticnet__alpha': [0.01, 0.1, 1.0, 10.0, 50.0],
                         'elasticnet__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]},
             scoring='neg_root_mean_squared_error', verbose=2)

In [454]:
search_en.best_params_

{'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.7}

In [455]:
search_en.best_score_

np.float64(-32541.274074893463)

In [456]:
best_rmse_cv = search_en.best_score_
best_rmse_cv

np.float64(-32541.274074893463)

In [457]:
best_model = search_en.best_estimator_

pred_test = best_model.predict(X_test)

rmse_test = np.sqrt(mean_squared_error(y_test, pred_test))
rmse_test

/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


np.float64(31423.298157239515)

In [458]:
best_model.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x12f3884c0>),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x12f397220>)])),
                ('elasticnet',
                 ElasticNet(alpha=0.1, l1_ratio=0.7, max_iter=20000,
                            random_state=42))])

In [459]:
test_preds = best_model.predict(test_df)
test_preds

/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


array([103074.2244981 , 150032.12686868, 172987.49256041, ...,
       158057.84180742, 100006.50140627, 232700.63970291])

In [460]:
test_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [461]:
test_df.shape

(1459, 80)

In [462]:
submission = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': test_preds
})

submission.head()

,Id,SalePrice
0,1461,103074.224498
1,1462,150032.126869
2,1463,172987.492560
3,1464,196000.233361
4,1465,191702.603459


In [464]:
import os

os.makedirs('submissions', exist_ok=True)

In [465]:
submission.to_csv('submissions/elasticnet_submission.csv', index=False)

In [466]:
test_preds = best_model.predict(test_df)

/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/kubrademirhan/Documents/kaggle_house_prices/.venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


In [467]:
submission = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': test_preds
})

In [468]:
submission.columns

Index(['Id', 'SalePrice'], dtype='object')

In [469]:
submission.head()

,Id,SalePrice
0,1461,103074.224498
1,1462,150032.126869
2,1463,172987.492560
3,1464,196000.233361
4,1465,191702.603459


In [470]:
submission.to_csv('submission.csv', index=False)

In [471]:
check = pd.read_csv('submission.csv')
print(check.columns)
print(check.head())

Index(['Id', 'SalePrice'], dtype='object')
     Id      SalePrice
0  1461  103074.224498
1  1462  150032.126869
2  1463  172987.492560
3  1464  196000.233361
4  1465  191702.603459


In [472]:
submission = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': test_preds
})

submission = submission[['Id', 'SalePrice']]

submission.to_csv('submission.csv', index=False)
print('saved:', submission.shape)
print(submission.head)

saved: (1459, 2)
<bound method NDFrame.head of         Id      SalePrice
0     1461  103074.224498
1     1462  150032.126869
2     1463  172987.492560
3     1464  196000.233361
4     1465  191702.603459
...    ...            ...
1454  2915   68941.169722
1455  2916   71448.035600
1456  2917  158057.841807
1457  2918  100006.501406
1458  2919  232700.639703

[1459 rows x 2 columns]>


In [473]:
pd.read_csv('submission.csv').columns

Index(['Id', 'SalePrice'], dtype='object')